In [ ]:
import pyspark
import pandas as pd
from pyarrow import fs, hdfs
from pyspark.sql.functions import col, split, explode
from pyspark.sql import SparkSession

hdfs.HadoopFileSystem("hdfs://namenode:9000")
conexao = fs.HadoopFileSystem(host="namenode", port=9000)

In [ ]:

conexao.get_file_info(fs.FileSelector('/', recursive = True))

In [ ]:

conexao.create_dir('smtv2')

In [ ]:
origem = '/home/joao.seje/Downloads/sentiment140.csv'

destino = '/smtv2/sentiment140.csv'

In [ ]:
with conexao.open_output_stream(destino) as stream:
    stream.write(open(origem, 'rb').read())

In [ ]:
with conexao.open_output_stream(destino) as stream:
    with open(origem, 'rb') as f:
        stream.write(f.read())

In [ ]:

with conexao.open_input_stream(destino) as f:
    print(f.read(50))


In [ ]:
spark = SparkSession.builder.appName('Spark').config('spark.sql.legacy.timeParserPolicy', 'LEGACY').getOrCreate()


In [ ]:
df_spark_hdfs = spark.read.csv('hdfs://namenode:9000/Somativa2', header = True, inferSchema = True, sep = ';')

In [ ]:
df_palavras = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('palavra'))


df_palavra_counts = df_palavras.filter(col('palavra') != '').groupBy('palavra').count().orderBy(col('quantidade').desc())

df_palavra_counts.show(10)



In [ ]:
df_palavras = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('palavra'))


df_mentions = df_palavras.filter(col('palavra').startswith('@')).groupBy('palavra').count().orderBy(col('quantidade').desc())


df_mentions.show(10)

In [ ]:
df_palavras = df_spark_hdfs.select(explode(split(col('text'), ' ')).alias('palavra'))


df_hashtags = df_palavras.filter(col('palavra').startswith('#')).groupBy('palavra').count().orderBy(col('quantidade').desc())

df_hashtags.show(10)